In [3]:
import konlpy
import pickle
import re

from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Twitter
hannanum = Hannanum()
kkma = Kkma()
twitter = Twitter()

import xgboost
import lightgbm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

---

# Train (역삼동 - 피자&양식)

In [4]:
location = '역삼동'
category = '피자&양식'

In [5]:
train_df = pickle.load(open('./data/{}_{}_df.pkl'.format(location, category),'rb'))
train_df.index += 1
train_df['Total'] = train_df['Total'].astype('int')

FileNotFoundError: [Errno 2] No such file or directory: './data/역삼동_피자&양식_df.pkl'

In [4]:
train_df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
9928,강남929치킨피자,dp**님,크리스피 순살/1,Delcious! And very impressed the delivery driv...,5,5,5,5,2018년 4월 28일
9929,강남929치킨피자,pe**님,세트6（슈퍼슈프림＋크리스피후라이드）/1(메뉴 변경 선택(변경 안함)),진짜 맛있네요 치킨 바삭하고 간 적당하고 느끼하지도 않고 피자도 딱히 흠잡을 곳이 ...,5,5,5,5,2018년 4월 27일
9930,강남929치킨피자,da**님,슈퍼슈프림피자/1,피자 시켰는데 배달 예정시간보다 일찍 왔고 일단 피자가 그림에서처럼 팬피자는 아니고...,4,5,4,5,2018년 4월 25일
9931,강남929치킨피자,wk**님,슈퍼슈프림피자/1,너무 배고파서 피자를 주문! 강남929 란 처음보는 피자배달 배달도 빨리오고 가격도...,5,5,5,5,2018년 4월 25일
9932,강남929치킨피자,ci**님,크리스피후라이드/1,퇴근 후 먹는 치킨 배달이 이렇게 빠를 줄이야..요즘 배달 다들 늦는데 이렇게빠르게...,5,5,5,5,2018년 4월 25일


In [5]:
train_df['Total'].value_counts()

5    6113
4    2226
3     823
1     511
2     259
Name: Total, dtype: int64

In [6]:
train_df.loc[train_df['Total'] == 2, 'Total'] = 1

In [7]:
train_df.loc[train_df['Total'] == 4, 'Total'] = 5

In [8]:
train_df['Total'].value_counts()
train_df['Total'].value_counts()

5    8339
3     823
1     770
Name: Total, dtype: int64

## hannanum

## twitter

In [9]:
hannanum.nouns(train_df['Review'][1])

['닭뼈', '비린내', '살고', '못먹어요', '별', '실망']

In [10]:
twitter.nouns(train_df['Review'][1])

['닭뼈', '비린', '별로', '실망']

In [11]:
hannanum.morphs(train_df['Review'][1])

['닭뼈',
 '에서',
 '비린내',
 '이',
 '나',
 '요',
 '그래서',
 '살고',
 '이',
 '기',
 '까지',
 '다',
 '못먹어요',
 '아무튼',
 '별',
 '로이',
 '에요',
 '실망',
 '이',
 'ㅂ니다']

In [12]:
twitter.morphs(train_df['Review'][1])

['닭뼈',
 '에서',
 '비린',
 '내',
 '나요',
 '그래서',
 '살',
 '고',
 '기까지',
 '다',
 '못',
 '먹어',
 '요',
 '아무튼',
 '별로',
 '에요',
 '실망',
 '입니',
 '다']

---

# Test (방이2동 - 치킨)

In [22]:
location = '방이2동'
category = '치킨'

In [23]:
test_df = pickle.load(open('./data/{}_{}_df.pkl'.format(location, category),'rb'))
test_df.index += 1
test_df['Total'] = test_df['Total'].astype('int')

In [24]:
test_df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
8869,또래오래-송파점,손님,,빠른배달 참좋아요~후라이드완전맛나고 이제치킨은 항상여기서 먹는걸로!,5,,,,2014년 4월 12일
8870,또래오래-송파점,손님,,맛있게 잘먹었어여 ㅎㅎㅎ 배달도빠르고 굿,5,,,,2014년 3월 8일
8871,또래오래-송파점,손님,,또래오래맛이에요,4,,,,2013년 12월 27일
8872,또래오래-송파점,yu**님,,ㅁ,5,,,,2013년 1월 13일
8873,또래오래-송파점,yu**님,,방이동 근처 배달가능한 치킨집중 최고임.,5,,,,2013년 1월 13일


In [25]:
test_df['Total'].value_counts()

5    5612
4    2024
3     662
1     342
2     233
Name: Total, dtype: int64

In [26]:
test_df.loc[test_df['Total'] == 2, 'Total'] = 1

In [27]:
test_df.loc[test_df['Total'] == 4, 'Total'] = 5

In [28]:
test_df['Total'].value_counts()
test_df['Total'].value_counts()

5    7636
3     662
1     575
Name: Total, dtype: int64

---

## Naive Bayes Model

In [114]:
X = train_df['Review']
len(X)

9932

In [115]:
y = train_df['Total']
len(y)

9932

In [35]:
MNB_model = Pipeline([
            ('vect', CountVectorizer()), 
            ('mb', MultinomialNB()),
        ])

In [116]:
%time MNB_model.fit(X, y)

CPU times: user 143 ms, sys: 6.11 ms, total: 150 ms
Wall time: 149 ms


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [117]:
X_test = test_df['Review']
len(X_test)

8873

In [118]:
y_test = test_df['Total']
len(y_test)

8873

In [39]:
print(accuracy_score(y_test, MNB_model.predict(X_test)))

0.8726473571509072


In [40]:
print(confusion_matrix(y_test, MNB_model.predict(X_test)))

[[ 122    6  447]
 [  19    6  637]
 [  16    5 7615]]


In [41]:
print(classification_report(y_test, MNB_model.predict(X_test)))

             precision    recall  f1-score   support

          1       0.78      0.21      0.33       575
          3       0.35      0.01      0.02       662
          5       0.88      1.00      0.93      7636

avg / total       0.83      0.87      0.83      8873



In [29]:
result = pd.DataFrame(MNB_model.predict(X_test), y_test).reset_index()
result.columns = ['Pred','Real']
# result[result['Pred']!=result['Real']]

---

## SVC

In [30]:
C = 1e20

SVC_rbf_model = Pipeline([
            ('vect', CountVectorizer()), 
            ('svc', SVC(kernel="rbf", C=C)),
        ])

In [ ]:
%time SVC_rbf_model.fit(X, y)

In [ ]:
print(accuracy_score(y_test, SVC_rbf_model.predict(X_test)))

In [ ]:
print(confusion_matrix(y_test, SVC_rbf_model.predict(X_test)))

In [ ]:
print(classification_report(y_test, SVC_rbf_model.predict(X_test)))

---

## XGB

In [308]:
XGB_model = Pipeline([
            ('vect', CountVectorizer()), 
            ('xgb', xgboost.XGBClassifier(max_depth=30, n_estimators=300, n_jobs=6)),
        ])

In [309]:
%time XGB_model.fit(X, y)

CPU times: user 1min 2s, sys: 288 ms, total: 1min 2s
Wall time: 1min 2s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [222]:
print(accuracy_score(y_test, XGB_model.predict(X_test)))

0.871858446973966


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [223]:
print(confusion_matrix(y_test, XGB_model.predict(X_test)))

[[ 195   48  332]
 [  48   72  542]
 [  70   97 7469]]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [224]:
print(classification_report(y_test, XGB_model.predict(X_test)))

             precision    recall  f1-score   support

          1       0.62      0.34      0.44       575
          3       0.33      0.11      0.16       662
          5       0.90      0.98      0.93      7636

avg / total       0.84      0.87      0.85      8873



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


---